In [1]:
from glob import glob
import json

In [2]:
files = sorted(glob('function-call/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

62401

In [3]:
def check_function(s):
    name = s.get('name', s.get('nama'))
    if name is None:
        return
    if 'parameters' not in s:
        return
    if 'properties' not in s['parameters']:
        return

    properties = set(s['parameters']['properties'])

    success = False
    required = s['parameters'].get('required', s['parameters'].get('diperlukan'))
    if required:
        required = set(required)
        if len(required & properties) == len(required):
            success = True
    else:
        success = True
    
    if success:
        return name, properties
    
def check_functions(function_calls):
    functions = {}

    if isinstance(function_calls, dict):
        o = check_function(data['e']['function_call'])
        if o is not None:
            functions[o[0]] = o[1]

    elif isinstance(function_calls, str):

        try:
            for s in function_calls.split('\n\n'):
                try:
                    s = eval(s)
                except:
                    continue

                o = check_function(s)
                if o is not None:
                    functions[o[0]] = o[1]
        except:
            s = eval(function_calls)
            if isinstance(s, dict):
                s = [s]
            for s_ in s:
                o = check_function(s_)
                if o is not None:
                    functions[o[0]] = o[1]
                
    return functions
    
rejected = []
with open('function-calls.jsonl', 'w') as fopen_l:
    for f in files:
        try:
            with open(f) as fopen:
                data = json.load(fopen)

            rs = [data['e']]
            rs.extend(data['rs'])

            for r in rs:

                try:
                    function_calls = r['function_call']
                    functions = check_functions(function_calls)

                    if not len(functions):
                        continue
                    
                    conversations = r.get('conversations', r.get('perbualan'))
                    if len(conversations) % 2 != 0:
                        conversations = conversations[:-1]

                    pairs = []
                    for i in range(0, len(conversations), 2):
                        try:
                            splitted = conversations[i + 1].get('content', conversations[i + 1].get('kandungan'))
                            if splitted is None:
                                continue
                            if '> ' in splitted:
                                splitted = splitted.split('> ')
                                splitted = splitted[1]
                            
                            splitted = splitted.replace('true', 'True').replace('false', 'False')
                            
                            failed = True
                            try:
                                splitted = eval(splitted)
                                failed = False
                            except:
                                pass
                            
                            if failed:
                                try:
                                    splitted = eval(splitted + '}')
                                    failed = False
                                except:
                                    pass
                            
                            if failed:
                                try:
                                    splitted = eval(splitted.replace("'{", '{') + '}')
                                except:
                                    continue
                                    
                            # print('spllitted', splitted, splitted['name'])
                            
                            name = splitted.get('name', splitted.get('nama'))
                            arguments = splitted.get('arguments', splitted.get('argument', splitted.get('argumen')))
                            if arguments is None:
                                arguments = splitted.get('hujah', splitted.get('hujahan'))
                            if arguments is None:
                                arguments = splitted.get('arahan', splitted.get('parameter'))
                            
                            if name not in functions:
                                continue
                                
                            if isinstance(arguments, str):
                                arguments = eval(arguments)
                                
                            arguments_keys = set(arguments.keys())
                            if len(functions[name] & arguments_keys) == len(arguments_keys):
                                splitted['arguments'] = arguments
                                c = [
                                    {
                                        'role': 'user',
                                        'content': conversations[i]['content'],
                                    },
                                    {
                                        'role': 'assistant',
                                        'content': f'<functioncall> {str(splitted)}'
                                    }
                                    ]
                                pairs.extend(c)

                        except Exception as e_splitted:
                            # print('e_splitted', e_splitted, conversations[i + 1], splitted)
                            pass

                    if function_calls and len(pairs):
                        d = {
                            'function_call': function_calls,
                            'conversations': pairs,
                        }
                        fopen_l.write(f'{json.dumps(d)}\n')
                        fopen_l.flush()
                except Exception as e_loop:
                    print('e_loop', e_loop, r)
                    rejected.append((e_loop, r))

        except Exception as e:
            print(e, data)

e_loop invalid syntax (<string>, line 81) {'function_call': '{\n "name": "create_invoice",\n "description": "Membuat invois untuk pelanggan",\n "parameters": {\n "type": "objek",\n\n"properties": {\n\n"customer_name": {\n\n"type": "string",\n\n"description": "Nama pelanggan"\n\n},\n\n"items": {\n\n"type": "array",\n\n"items": {\n\n"type": "objek",\n\n"properties": {\n\n"name": {\n\n"type": "string",\n\n"description": "Nama item"\n\n},\n\n"quantity": {\n\n"type": "integer",\n\n"description": "Kuantiti item"\n\n},\n\n"price": {\n\n"type": "number",\n\n"description": "Harga item"\n\n}\n\n},\n\n"required": [\n\n"name",\n\n"quantity",\n\n"price"\n\n]\n\n}\n\n}\n\n},\n\n"required": [\n\n"customer_name",\n\n"items"\n\n]\n\n}\n\n}\n\n{\n\n"name": "calculate_distance",\n\n"description": "Mengira jarak antara dua lokasi",\n\n"parameters": {\n\n"type": "objek",\n\n"properties": {\n\n"origin": {\n\n"type": "string",\n\n"description": "Lokasi permulaan"\n\n},\n\n"destination": {\n\n"type": "string"

In [4]:
!wc -l function-calls.jsonl

179450 function-calls.jsonl


In [5]:
ls = []
with open('function-calls.jsonl') as fopen:
    for l in fopen:
        ls.append(json.loads(l))
        if len(ls) > 10:
            break

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='function-calls.jsonl',
    path_in_repo='function-calls.jsonl',
    repo_id='mesolitica/chatgpt-malay-function-call',
    repo_type='dataset',
)

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


function-calls.jsonl:   0%|          | 0.00/200M [00:00<?, ?B/s]